In [1]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/캡스톤/data/Anime3/'

In [5]:
with open(path+"train_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open(path+"valid_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open(path+"test_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [6]:
with open(path+"train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open(path+"valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open(path+"test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

## Model

In [7]:
!pip install keras_multi_head
#!pip install keras_self_attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14993 sha256=76eac5d85f9491036dc176d82362b021520a088cf52bc3d588945435162498bc
  Stored in directory: /root/.cache/pip/wheels/86/aa/3c/9d15d24005179dae08ff291ce99c754b296347817d076fd9fb
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=05d7e3d5c0a3bde03e5cd41b4e84045fbe540e398e1ab921f7371ee9b9695edb
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-multi-head keras-self-attention


In [8]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint
from keras_multi_head import MultiHead
#from keras_self_attention import SeqSelfAttention as Attention

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 3
#current_path ='data/'
#local_path = 'data/'

In [9]:
# set parameters:
# V = 142381
# V = 226928
V = 31466 # 48456 #  33531 6158 48456
embedding_dim = 50
max_len = 1253 # 1 #  1945 77 1
filters_num = 64
kernel_size = 3

In [10]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))
input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu',
             #    padding='same'
             )
maxpool1d = MaxPool1D(max_len-2)
batchnorm1d = BatchNormalization()
dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(MultiHead(conv1d))(input_vector)
validation_conv = MultiHead(conv1d)(validation_vector)
convolutional_vector = tf.keras.backend.squeeze(convolutional_vector, axis=4)
validation_conv = tf.keras.backend.squeeze(validation_conv, axis=3)

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_vector)
validation_maxpooling = maxpool1d(validation_conv)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [11]:
print(input_vector.shape, validation_vector.shape,
      convolutional_vector.shape, validation_conv.shape,
      maxpooling_vector.shape, validation_maxpooling.shape,
      middle_output.shape, middle_validation.shape)

(None, 3, 1253, 50) (None, 1253, 50) (None, 3, 1251, 64) (None, 1251, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 15) (None, 1, 15)


In [12]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [13]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [14]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', 'mape'])

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1253)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1253)]       0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 3, 1253, 50)  1573300    ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1253, 50)     1573300     ['input_2[0][0]']            

In [16]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 61s 186ms/step - loss: 5.6730 - mse: 5.6730 - mae: 1.6015 - mape: 24.1300 - val_loss: 2.4833 - val_mse: 2.4833 - val_mae: 1.1565 - val_mape: 20.9580
Epoch 2/30
256/256 [==============================] - 48s 187ms/step - loss: 2.3089 - mse: 2.3089 - mae: 1.1602 - mape: 19.5822 - val_loss: 2.5032 - val_mse: 2.5032 - val_mae: 1.1360 - val_mape: 21.0254
Epoch 3/30
256/256 [==============================] - 49s 190ms/step - loss: 2.2762 - mse: 2.2762 - mae: 1.1491 - mape: 19.4380 - val_loss: 2.3864 - val_mse: 2.3864 - val_mae: 1.1821 - val_mape: 20.6047
Epoch 4/30
256/256 [==============================] - 49s 190ms/step - loss: 2.2603 - mse: 2.2603 - mae: 1.1439 - mape: 19.3397 - val_loss: 2.3877 - val_mse: 2.3877 - val_mae: 1.1438 - val_mape: 20.6190
Epoch 5/30
256/256 [==============================] - 49s 190ms/step - loss: 2.2371 - mse: 2.2371 - mae: 1.1369 - mape: 19.2318 - val_loss: 2.3578 - val_mse: 2.3578 - val_mae: 1.1682 - val

In [17]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 9s 4ms/step - loss: 2.2262 - mse: 2.2262 - mae: 1.1191 - mape: 19.0338
[2.2262423038482666, 2.2262423038482666, 1.1190720796585083, 19.03379249572754]


In [ ]:
# model.save(path+"IMDB_ADLFM_Multi_head_conv1d")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/캡스톤/data/IMDB_ADLFM_Multi_head_conv1d/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/캡스톤/data/IMDB_ADLFM_Multi_head_conv1d/assets
